In [25]:
import torch
SEED = 123
torch.manual_seed(SEED) 

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# from torchsummary import summary
from torchinfo import summary
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import warnings
from tqdm import tqdm
from datetime import datetime
warnings.filterwarnings('ignore')

from PIL import Image
import os
import glob

# check OS is Window or Mac
import platform
device = torch.device("cpu")

if platform.system() == 'Windows':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
elif platform.system() == 'Darwin':
    try:
        device = torch.device("mps")
    except:
        device = torch.device("cpu")

float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind': float_formatter})

print(device)

cpu


In [70]:
class CustomFCNetwork(nn.Module):
    def __init__(self):
        super(CustomFCNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(19200, 128)  # Update fc1 input size to match the expected size
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 60)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

class LeNet5V1(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature = nn.Sequential(
            # 1
            nn.Conv2d(in_channels=3,    # cantidad de canales RGB == 3
                      out_channels=6, 
                      kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            # 2
            nn.Conv2d(in_channels=6, 
                      out_channels=16,
                      kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16*22*22, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.classifier(self.feature(x))

In [81]:
try:
    modelFCN = CustomFCNetwork()
    model_name = "FCN_final"
    modelFCN.load_state_dict(torch.load(f"models/best_model_{model_name}.pth", map_location=torch.device('cpu')))
    modelFCN.eval()  # Set model to evaluation mode after loading
except Exception as e:
    print(f"Error: {e}")


In [32]:
try:
    modelCNN = LeNet5V1()
    model_name = "LeNet5_final"
    modelCNN.load_state_dict(torch.load(f"models/best_model_{model_name}.pth", map_location=torch.device('cpu')))
    modelCNN.eval()  # Set model to evaluation mode after loading
except Exception as e:
    print(f"Error: {e}")


## Adversarial Training

In [92]:
dataset_path = './img/'
batch_size = 8 
transform = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
])

train_dataset = ImageFolder(root='./data_split/train', transform=transform)
val_dataset = ImageFolder(root='./data_split/test', transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [84]:
# generar adversial images
def fgsm_attack(data, target, model, epsilon=0.03):
    data.requires_grad = True
    output = model(data)
    loss = F.cross_entropy(output, target)
    model.zero_grad()
    loss.backward()
    perturbed_data = data + epsilon * data.grad.sign()
    perturbed_data = torch.clamp(perturbed_data, 0, 1)  # Ensure pixel values are within valid range
    return perturbed_data

def adversarial_training(model, model_name):
    # training loop
    num_epochs = 10
    epsilon = 0.03  #magnitud de la pertubación

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_dataloader:
            images, labels = images.to(device), labels.to(device)
            
            # genera adversarial images
            adv_images = fgsm_attack(images, labels, model, epsilon)
                        
            optimizer.zero_grad()
            outputs = model(torch.cat((images, adv_images), dim=0))
            combined_labels = torch.cat((labels, labels), dim=0)
            loss = criterion(outputs, combined_labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = outputs.max(1)
            total += combined_labels.size(0)
            correct += predicted.eq(combined_labels).sum().item()
        
        # Print statistics
        epoch_loss = running_loss / len(train_dataloader)
        accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

        if (epoch == 8):  # Adjust the frequency of saving checkpoints as needed
            checkpoint_path = "./models/" + f'adversarial_{model_name}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                # Add other relevant training parameters or state
            }, checkpoint_path)


In [94]:
adversarial_training(modelFCN, "FCN")

In [ ]:
adversarial_training(modelCNN,"LeNet5")